<a href="https://colab.research.google.com/github/galdir/classificacao_com_bert/blob/main/classificacao_de_textos_com_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Classificação de Textos usando BERT

# Instalação das bibliotecas necessárias


In [3]:
# Descomente e execute se necessário
#!pip install transformers pandas numpy scikit-learn torch datasets

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

# Importação das bibliotecas


In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Verificar se GPU está disponível


In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando {device}")

Usando cuda


# Baixar e preparar o dataset B2W-Reviews01

In [3]:
print("Baixando o dataset B2W-Reviews01...")
url = "https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/refs/heads/main/B2W-Reviews01.csv"
df = pd.read_csv(url, low_memory=False)

Baixando o dataset B2W-Reviews01...


In [4]:
df.head()

,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958.0,F,RJ
1,2018-01-01 00:13:48,014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...,22562178,Copo Acrílico Com Canudo 500ml Rocie,NaN,Utilidades Domésticas,"Copos, Taças e Canecas","Preço imbatível, ótima qualidade",4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ...",1996.0,M,SC
2,2018-01-01 00:26:02,44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...,113022329,Panela de Pressão Elétrica Philips Walita Dail...,philips walita,Eletroportáteis,Panela Elétrica,ATENDE TODAS AS EXPECTATIVA.,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1984.0,M,SP
3,2018-01-01 00:35:54,ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...,113851581,Betoneira Columbus - Roma Brinquedos,roma jensen,Brinquedos,Veículos de Brinquedo,presente mais que desejado,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1985.0,F,SP
4,2018-01-01 01:00:28,7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...,131788803,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",lg,TV e Home Theater,TV,"Sem duvidas, excelente",5,Yes,"A entrega foi no prazo, as americanas estão de...",1994.0,M,MG


# Preparar os dados - apenas textos e sentimentos


In [5]:
df = df[['review_text', 'overall_rating']]
df.columns = ['texto', 'nota']

In [6]:
df.head(10)

,texto,nota
0,Estou contente com a compra entrega rápida o ú...,4
1,"Por apenas R$1994.20,eu consegui comprar esse ...",4
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,4
3,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,4
4,"A entrega foi no prazo, as americanas estão de...",5
5,"Excelente produto, por fora em material acríli...",5
6,"produto mto bom, com essa garrafinha vc pode a...",5
7,Produto excelente qualidade boa câmera desenvo...,4
8,O barulho e minimo e o vento é bem forte na ve...,5
9,MEU PRODUTO NAO FOI ENTREGUE E A AMERICANAS ES...,1


# Converter notas para sentimento binário (0-negativo, 1-positivo)


In [7]:
df = df[(df['nota'] <= 2) | (df['nota'] >= 4)].copy()

In [8]:
df['sentimento'] = df['nota'].apply(lambda x: 1 if x >= 4 else 0)

In [9]:
df = df[['texto', 'sentimento']]

In [10]:
df = df[df['texto'].notna() & (df['texto'] != '')].reset_index(drop=True)

In [11]:
df.head(10)

,texto,sentimento
0,Estou contente com a compra entrega rápida o ú...,1
1,"Por apenas R$1994.20,eu consegui comprar esse ...",1
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1
3,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1
4,"A entrega foi no prazo, as americanas estão de...",1
5,"Excelente produto, por fora em material acríli...",1
6,"produto mto bom, com essa garrafinha vc pode a...",1
7,Produto excelente qualidade boa câmera desenvo...,1
8,O barulho e minimo e o vento é bem forte na ve...,1
9,MEU PRODUTO NAO FOI ENTREGUE E A AMERICANAS ES...,0


# Verificar distribuição


In [12]:
print(f"Total de exemplos: {len(df)}")
print(f"Exemplos positivos: {sum(df['sentimento'])}")
print(f"Exemplos negativos: {len(df) - sum(df['sentimento'])}")

Total de exemplos: 113088
Exemplos positivos: 79316
Exemplos negativos: 33772


# Limitar dataset para processamento mais rápido (opcional)


In [13]:
# Usando apenas 1000 exemplos balanceados para demonstração
neg = df[df['sentimento'] == 0].sample(500, random_state=42)
pos = df[df['sentimento'] == 1].sample(500, random_state=42)
df = pd.concat([neg, pos]).sample(frac=1, random_state=42).reset_index(drop=True)

In [14]:
df.shape

(1000, 2)

# Separar em treino e teste

In [15]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


# Carregar o Tokenizador do modelo BERT pré-treinado para português

In [16]:
model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

# Função de pré-processamento para tokenização


In [17]:
exemplo = train_df.iloc[0]["texto"]
exemplo

'Recomendo, excelente produto, de fácil manuseio e ideal para quem tem pouco tempo!'

In [18]:
tokenizer(exemplo, truncation=True, padding="max_length", max_length=128)

{'input_ids': [101, 2325, 3309, 214, 117, 9235, 3576, 117, 125, 6904, 8473, 1556, 122, 7503, 221, 1977, 376, 1695, 596, 106, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [19]:
def preprocess_function(examples):
    # Tokenizar os textos e definir truncation/padding
    tokenized = tokenizer(
        examples["texto"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

    # IMPORTANTE: Adicionar as etiquetas (labels)
    tokenized["labels"] = examples["sentimento"]

    return tokenized

In [20]:
preprocess_function(train_df.iloc[0])

{'input_ids': [101, 2325, 3309, 214, 117, 9235, 3576, 117, 125, 6904, 8473, 1556, 122, 7503, 221, 1977, 376, 1695, 596, 106, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Converter DataFrames para o formato Dataset da biblioteca datasets


In [21]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [22]:
train_dataset

Dataset({
    features: ['texto', 'sentimento', '__index_level_0__'],
    num_rows: 800
})

# Aplicar tokenização

In [23]:
train_tokenized = train_dataset.map(preprocess_function, batched=True)
test_tokenized = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [24]:
train_tokenized

Dataset({
    features: ['texto', 'sentimento', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 800
})

# Carregar modelo pretreinado e preparado para classificação

In [25]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Configurar argumentos de treinamento


In [26]:
training_args = TrainingArguments(
    output_dir="./resultados",
    num_train_epochs=1,            # Apenas 1 época para demonstração
    per_device_train_batch_size=8,
    learning_rate=2e-5,
)

# Função para cálculo das métricas

In [27]:
def compute_metrics(resultado):
    rotulos_verdadeiros = resultado.label_ids
    rotulos_preditos = resultado.predictions.argmax(-1)
    acc = (rotulos_preditos == rotulos_verdadeiros).mean()
    return {"accuracy": acc}

# Configurar o treinador


In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

# Treinar o modelo


In [29]:
print("\nIniciando treinamento...")
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.



Iniciando treinamento...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: galdir (ctai-ufba) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=100, training_loss=0.3150398826599121, metrics={'train_runtime': 539.1921, 'train_samples_per_second': 1.484, 'train_steps_per_second': 0.185, 'total_flos': 52622211072000.0, 'train_loss': 0.3150398826599121, 'epoch': 1.0})

# Avaliar o modelo


In [30]:
print("\nAvaliando o modelo...")
eval_results = trainer.evaluate()
print(f"Acurácia: {eval_results['eval_accuracy']:.4f}")


Avaliando o modelo...


Acurácia: 0.9550


# Função para classificar novos textos


In [34]:
def classificar_texto(texto):
    inputs = tokenizer(texto, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    pred = outputs.logits.argmax(dim=-1).item()
    sentimento = "Positivo" if pred == 1 else "Negativo"

    return sentimento

# Testar com novos exemplos

In [36]:
texto = "Produto excelente, entrega rápida e atendimento nota 10!"

print(f"\nTexto: {texto}")

sentimento = classificar_texto(texto)

print(f"Classificação: {sentimento}")


Texto: Produto excelente, entrega rápida e atendimento nota 10!
Classificação: Positivo


In [37]:
texto = "Péssima experiência, o produto veio quebrado e o suporte não resolveu."

print(f"\nTexto: {texto}")

sentimento = classificar_texto(texto)

print(f"Classificação: {sentimento}")


Texto: Péssima experiência, o produto veio quebrado e o suporte não resolveu.
Classificação: Negativo


In [40]:
texto = "eu recomendaria"

print(f"\nTexto: {texto}")

sentimento = classificar_texto(texto)

print(f"Classificação: {sentimento}")


Texto: eu recomendaria
Classificação: Positivo


In [39]:
texto = "eu desrecomendaria"

print(f"\nTexto: {texto}")

sentimento = classificar_texto(texto)

print(f"Classificação: {sentimento}")


Texto: eu desrecomendaria
Classificação: Negativo


In [38]:
texto = "normal, não supreendeu."

print(f"\nTexto: {texto}")

sentimento = classificar_texto(texto)

print(f"Classificação: {sentimento}")


Texto: normal, não supreendeu.
Classificação: Negativo
